In [21]:
from pdf2image import convert_from_path

In [22]:
pages = convert_from_path(r"/Users/yashpalande/Desktop/College_Work/DBDA_FINAL_PROJ/medical-project/backend/docs/patient_details/pd_1.pdf")
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1867x2000>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2000x1398>]

In [23]:
pages[0].show()

In [24]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/bin/tesseract'

In [25]:
text = pytesseract.image_to_string(pages[0],lang="eng")
print(text)

47/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Case of Emergency
m _ a _
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
. : a ee

Chicken Pox (Varicella):

IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine



In [26]:
import numpy as np
import cv2
from PIL import Image

def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
    processed_image = cv2.adaptiveThreshold(
        resized,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 
        61,
        11
    )
    return processed_image

In [27]:
img = preprocess_image(pages[0])
Image.fromarray(img).show()

In [28]:
pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/bin/tesseract'
text = pytesseract.image_to_string(img, lang='eng')
print(text)

17/12/2020

Patient Medical Record

Patient Information Birth Date

Kathy Crawford May 6 1972

(737) 988-0851 Weight’

9264 Ash Dr 95

New York City, 10005 '

United States Height:
190

In Case of Emergency
ee J
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
nn i
Chicken Pox (Varicella): Measies:
IMMUNE

IMMUNE
Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches}:

Migraine



In [30]:
text = '''17/12/2020

Patient Medical Record

Patient Information Birth Date

Kathy Crawford May 6 1972

(737) 988-0851 Weight’

9264 Ash Dr 95

New York City, 10005 '

United States Height:
190

In Case of Emergency
ee J
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
nn i
Chicken Pox (Varicella): Measies:
IMMUNE

IMMUNE
Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine'''

# Extracting Name :

In [31]:
import re

pattern = 'Patient Information(.*?)\(\d{3}\)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

[' Birth Date\n\nKathy Crawford May 6 1972\n\n']

In [32]:
matches[0].strip()

'Birth Date\n\nKathy Crawford May 6 1972'

In [33]:
match = matches[0].replace("Birth Date","").strip()
match

'Kathy Crawford May 6 1972'

In [34]:
pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'

date_matches = re.findall(pattern, match)
date = date_matches[0][0]
date

'May 6 1972'

In [35]:
match.replace(date, '').strip()

'Kathy Crawford'

In [36]:
def remove_noise_from_name(name):
    name = name.replace('Birth Date','').strip()
    date_pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    date_matches = re.findall(date_pattern, name)
    
    if date_matches:
        date = date_matches[0][0]
        name = name.replace(date, '').strip()
    
    return name

In [37]:
name = ' Birth Date\n\nKathy Crawford May 6 1972\n\n'

name = remove_noise_from_name(name)
name

'Kathy Crawford'

# Extracting Phone no :

In [38]:
pattern = 'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

[(' Birth Date\n\nKathy Crawford May 6 1972\n\n', '(737) 988-0851')]

In [39]:
matches[0][1]

'(737) 988-0851'

# Extracting Vaccine Info :

In [40]:
pattern = 'Have you had the Hepatitis B vaccination\?.*(Yes|No)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['No']

# Extracting Medical Problems :

In [41]:
pattern = 'List any Medical Problems \(asthma, seizures, headaches\):(.*)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'Migraine'